In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/diego_tools/labo3/")
os.chdir("/home/dcastro_contacto/buckets/b1/")
####################################################

In [ ]:
arch_temp = "datasets/externos/temperaturaMediaHistoricaCABA.xlsx"
arch_ipc = "datasets/externos/IPC.xlsx"
arch_dolar = "datasets/externos/cotizacionDolar.xlsx"
arch_salida = "datasets/emp3_exogenas.csv"

## Temperatura

In [ ]:
df_temp = pd.read_excel(arch_temp)
df_temp.info()

In [ ]:
# Para 2017 no esta la temperatura media, la aproximo
df_temp.temp_media = np.where(df_temp.temp_media.isna(),(df_temp.temp_max_media+df_temp.temp_min_media)/2,df_temp.temp_media)

In [ ]:
df_temp.head()

In [ ]:
df_temp.info()

In [ ]:
print(min(df_temp.periodo_fecha))
print(max(df_temp.periodo_fecha))

## IPC

In [ ]:
df_ipc = pd.read_excel(arch_ipc)
df_ipc.info()

In [ ]:
df_ipc.rename(columns={"fecha":"periodo_fecha"},inplace=True)

In [ ]:
df_ipc.head()

In [ ]:
print(min(df_ipc.periodo_fecha))
print(max(df_ipc.periodo_fecha))

## Dolar

In [ ]:
df_dolar = pd.read_excel(arch_dolar)
df_dolar.info()

In [ ]:
df_dolar.head()

In [ ]:
  def getYearMonth(d):
      return str(d.year) + str.rjust(str(d.month),2,"0")

In [ ]:
df_dolar['periodo'] = df_dolar['fecha'].apply(lambda x: getYearMonth(x))
df_dolar.periodo.unique()

In [ ]:
df_dolar_resum = df_dolar.groupby("periodo").mean(numeric_only=True).reset_index()
df_dolar_resum.rename(columns={"cotizacion_dolar_venta":"promedio_mens_dolar_venta"},inplace=True)
df_dolar_resum.promedio_mens_dolar_venta = round(df_dolar_resum.promedio_mens_dolar_venta,2)
df_dolar_resum['periodo_fecha'] = pd.to_datetime(df_dolar_resum['periodo'], format='%Y%m')
df_dolar_resum.info()

## Variable "Coyuntura pais"

In [ ]:
df_coyuntura_pais = pd.DataFrame({"periodo":["201908"],"catastrofe":[True]})
df_coyuntura_pais['periodo_fecha'] = pd.to_datetime(df_coyuntura_pais['periodo'], format='%Y%m')
df_coyuntura_pais.drop(columns="periodo",inplace=True)

df_coyuntura_pais.head()

In [ ]:
df_coyuntura_pais.info()

## Join de todo

In [ ]:
print(len(df_temp))
df_exogenas = pd.merge(df_temp,df_ipc,on="periodo_fecha",how="left")
df_exogenas = pd.merge(df_exogenas,df_dolar_resum,on="periodo_fecha",how="left")
df_exogenas = pd.merge(df_exogenas,df_coyuntura_pais,on="periodo_fecha",how="left")
print(len(df_exogenas))

In [ ]:
df_exogenas.info()

In [ ]:
df_exogenas.catastrofe = df_exogenas.catastrofe.fillna(False)
df_exogenas.drop(columns="periodo",inplace=True)
df_exogenas.info()

In [ ]:
df_exogenas.head()

In [ ]:
df_exogenas.tail(20)

In [ ]:
df_exogenas.to_csv(arch_salida,index=False)